# 데이터 전처리

In [21]:
from tqdm.notebook import tqdm
import pandas as pd
import json
import glob
import re

def load_json_files(path_pattern):
    files = glob.glob(path_pattern)
    data = []
    for file in tqdm(files, "Loading JSON files"):
        with open(file, 'r', encoding='utf-8-sig') as f:
            json_data = json.load(f)
            data.append(json_data)
    return data

def remove_highlights(text: str) -> str:
    """하이라이트를 제거하고 연속된 같은 문자를 하나로 축소하는 함수"""
    # 하이라이트 제거
    clean_text = re.sub(r"''(.*?)'''", "\\1", text.strip())
    
    # 연속된 같은 문자를 하나로 축소
    clean_text = re.sub(r'(.)\1+', r'\1', clean_text)

    return clean_text

def create_one_hot_encoding(data):
    """One-hot encoding DataFrame을 생성하는 함수"""
    # 기본 데이터프레임 생성
    df = pd.DataFrame(data)

    # 라벨 목록 (영어 대문자로 수정)
    label_categories = [
        "INSULT",  # 모욕
        "PROFANE",  # 욕설
        "OBSCENE",  # 외설
        "THREAT",  # 폭력위협/범죄조장
        "HATE_SPEECH",  # 성혐오
        "AGEISM",  # 연령
        "RACISM",  # 인종/지역
        "DISABILITY",  # 장애
        "RELIGION",  # 종교
        "POLITICAL",  # 정치성향
        "PROFESSION"  # 직업
    ]

    labels = [
        "모욕",
        "욕설",
        "외설",
        "폭력위협/범죄조장",
        "성혐오",
        "연령",
        "인종/지역",
        "장애",
        "종교",
        "정치성향",
        "직업"
    ]

    # One-hot encoding DataFrame 초기화
    one_hot_labels = pd.DataFrame(0, index=df.index, columns=label_categories, dtype='float32')

    # 'text' 하이라이트 처리 및 Labels 설정
    for index, row in tqdm(df.iterrows(), "One Hot Encoding", total=len(df)):
        # "문장" 열에서 하이라이트 제거
        df.loc[index, '문장'] = remove_highlights(row['문장'])
        
        for idx, category in enumerate(labels):
            if category in row and row[category] >= 1:
                one_hot_labels.at[index, label_categories[idx]] = row[category] * 0.125 + 0.5
    
    # 최종 DataFrame 생성
    saved = pd.concat([df[['문장']], one_hot_labels], axis=1)  # 'text' 열 포함
    saved.rename(columns={'문장': 'text'}, inplace=True)
    
    return saved

# JSON 파일 로드
json_files_path = 'data/selectstar/C*.json'
data = load_json_files(json_files_path)

# 원본 데이터로부터 One-hot encoding DataFrame 생성
one_hot_df = create_one_hot_encoding(data)

# 결과 출력 및 CSV 파일로 저장(optional)
print(one_hot_df.head(n=10))
one_hot_df.to_csv('selectstar-encoded.csv', encoding='utf-8', index=False)

Loading JSON files:   0%|          | 0/100000 [00:00<?, ?it/s]

One Hot Encoding:   0%|          | 0/100000 [00:00<?, ?it/s]

                                                text  INSULT  PROFANE  \
0  국민들은 깨어나야 합니다. 지금 민주당을 지지하는 사림들 보세요~ 종북주의자들이 판...   0.625    0.000   
1                           저 어른들...지금 정부 꼬라지 보는것같구만   0.750    0.000   
2  대깨문 출동 이번 코로나가 확산되는건 민노총 때문이라고 몰고가라~~~ 눼눼~ 출동출...   0.750    0.000   
3                                 아프간과 비교를...ㅎ 치매드셨나   0.750    0.000   
4  작년 국민집회때 #@이름#목사 감옥보내더니 ㅋㅋㅋ8000명 민노총은 집회허락하는 이...   0.625    0.000   
5  태권도는 졸라 깨지고~창피하다!!!!!!태권도 종주국 이라고 어디가서 떠들지 마라!...   0.750    0.625   
6  더불당은 동네 똥개 크게 소리 짓는 암숫개들 다 모아 놓았네 오늘 초복인데 도살장 ...   0.750    0.000   
7   미국이 지원해서 가장 성공한 케이스가 대한민국.그리고 잘살게되니까 반미하는 토종빨갱이들   0.750    0.000   
8                               세금 그렇게 먹으면 죽을만 하지 ㅇㅇ   0.625    0.000   
9  또다시 더불어당 #@이름# #@이름#이 입을 맞추며 분위기를 잡고 있다. 싹 쓸어버...   0.625    0.000   

   OBSCENE  THREAT  HATE_SPEECH  AGEISM  RACISM  DISABILITY  RELIGION  \
0      0.0     0.0          0.0     0.0     0.0       0.000       0.0   
1      0.0     0.0          0.0     0.0     0.0   

In [ ]:
from transformers import (
    AutoModelForSequenceClassification,
    AutoTokenizer,
    Trainer,
    TrainingArguments,
    DataCollatorWithPadding,
)
import torch
import numpy as np
import pandas as pd
from sklearn.metrics import classification_report, f1_score, hamming_loss
from sklearn.model_selection import train_test_split
from datasets import DatasetDict, Dataset
import os

# 레이블 설정 (영어로 의미 표현)
label_categories = [
    "INSULT",  # 모욕
    "PROFANE",  # 욕설
    "OBSCENE",  # 외설
    "THREAT",  # 폭력위협/범죄조장
    "HATE_SPEECH",  # 성혐오
    "AGEISM",  # 연령
    "RACISM",  # 인종/지역
    "DISABILITY",  # 장애
    "RELIGION",  # 종교
    "POLITICAL",  # 정치성향
    "PROFESSION"  # 직업
]

# label2id 및 id2label 정의
label2id = {label: i for i, label in enumerate(label_categories)}
id2label = {i: label for i, label in enumerate(label_categories)}

def load_and_preprocess_data():
    # 데이터 로드
    raw_dataset = pd.read_csv('./selectstar-encoded.csv')

    # 레이블을 One-hot Encoding 형식으로 변환하기
    def create_labels(row):
        return [1 if row[category] > 0 else 0 for category in label_categories]

    # 레이블 데이터 생성
    raw_dataset['labels'] = raw_dataset.apply(create_labels, axis=1)

    # 데이터 분할
    X_train, X_eval, Y_train, Y_eval = train_test_split(
        raw_dataset['문장'].tolist(), 
        raw_dataset['labels'].tolist(), 
        test_size=0.2, 
        random_state=42,  # 재현성을 위해 시드 고정 
        stratify=raw_dataset['labels']  # 레이블에 따라 stratified 분할
    )

    return X_train, X_eval, Y_train, Y_eval

if __name__ == "__main__":
    # CUDA, MPS, CPU 체크
    device = torch.device('cuda' if torch.cuda.is_available() else ('mps' if torch.backends.mps.is_available() else 'cpu'))
    print(f"Using device: {device}")

    # 모델과 토크나이저 로드
    tokenizer = AutoTokenizer.from_pretrained("beomi/KcELECTRA-base", trust_remote_code=True)
    model = AutoModelForSequenceClassification.from_pretrained(
        "beomi/KcELECTRA-base",
        num_labels=len(label_categories),  # 레이블 수
        id2label=id2label,  # id2label 설정
        label2id=label2id,  # label2id 설정
        problem_type="multi_label_classification"
    ).to(device)
    
    # 데이터셋 로드 및 변환
    X_train, X_eval, Y_train, Y_eval = load_and_preprocess_data()

    # Tokenization 및 인덱싱
    train_encodings = tokenizer(X_train, padding=True, truncation=True, return_tensors="pt")
    eval_encodings = tokenizer(X_eval, padding=True, truncation=True, return_tensors="pt")

    # PyTorch Dataset 정의
    class MultiLabelDataset(torch.utils.data.Dataset):
        def __init__(self, encodings, labels):
            self.encodings = encodings
            self.labels = labels
            
        def __getitem__(self, idx):
            item = {key: val[idx] for key, val in self.encodings.items()}
            item['labels'] = torch.tensor(self.labels[idx], dtype=torch.float)  # 멀티 레이블인 경우 float형으로 변경
            return item

        def __len__(self):
            return len(self.labels)

    # 데이터셋 생성
    train_dataset = MultiLabelDataset(train_encodings, Y_train)
    eval_dataset = MultiLabelDataset(eval_encodings, Y_eval)

    # Trainer 설정
    training_args = TrainingArguments(
        output_dir='./results',
        num_train_epochs=5,
        per_device_train_batch_size=16,
        per_device_eval_batch_size=16,
        logging_dir='./logs',
        logging_steps=100,
        evaluation_strategy="epoch",
        save_strategy="steps",
        save_total_limit=3,
        save_steps=1000,
        learning_rate=5e-5,
        fp16=torch.cuda.is_available(),  # FP16 활성화 GPU의 경우
    )

    # Trainer 정의
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
    )

    # 모델 학습
    trainer.train()

    # 모델 평가
    eval_results = trainer.evaluate()
    print(eval_results)

    # 테스트 데이터셋 평가
    predictions = trainer.predict(eval_dataset)
    pred_labels = (torch.sigmoid(torch.tensor(predictions.predictions)) >= 0.5).cpu().numpy()
    
    test_labels_np = np.vstack(eval_dataset.labels)

    # Classification report 출력
    print(classification_report(test_labels_np, pred_labels, target_names=label_categories))

    # 모델 및 토크나이저 저장
    model_save_path = "./saved_model"
    os.makedirs(model_save_path, exist_ok=True)

    trainer.save_model(model_save_path)
    tokenizer.save_pretrained(model_save_path)

    print(f"Model and tokenizer saved to {model_save_path}")